In [1]:
import streamlit as st 
import random
import time
from dotenv import load_dotenv
import os
import langchain
from langchain_groq import ChatGroq
from langchain_core.messages import HumanMessage,AIMessage
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationChain
from langchain_core.runnables.history import RunnableWithMessageHistory

from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import START, MessagesState, StateGraph



load_dotenv()
api_key=os.getenv("GROQ_API_KEY")

model = ChatGroq(model="llama3-8b-8192")



# memory=ConversationBufferMemory()
# conversation = ConversationChain(llm=model, memory=memory)

# print(model.invoke([HumanMessage(content="Hi! I'm Bob")]))
# print(model.invoke([HumanMessage(content="What's my name?")]))

In [3]:
# Define a new graph
workflow = StateGraph(state_schema=MessagesState)


# Define the function that calls the model 
def call_model(state: MessagesState):
    response = model.invoke(state["messages"])
    return {"messages": response}


# Define the (single) node in the graph
workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

# Add memory
memory = MemorySaver()
app = workflow.compile(checkpointer=memory)



In [4]:
config = {"configurable": {"thread_id": "abc123"}}

query = "Hi! I'm Bob."

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()  # output contains all messages in state

================================== Ai Message ==================================

Hi Bob! It's nice to meet you. Is there something I can help you with or would you like to chat?


In [5]:
query = "What's my name?"

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

Your name is Bob!


In [6]:
config = {"configurable": {"thread_id": "ab4"}}

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

I'm happy to help! However, I'm a large language model, I don't have the ability to know your name unless you tell me. Could you please introduce yourself?


In [7]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt_template = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You talk like a pirate. Answer all questions to the best of your ability.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

In [8]:
workflow = StateGraph(state_schema=MessagesState)


def call_model(state: MessagesState):
    prompt = prompt_template.invoke(state)
    response = model.invoke(prompt)
    return {"messages": response}


workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

In [9]:
config = {"configurable": {"thread_id": "abc345"}}
query = "Hi! I'm Jim."

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

Arrrr, shiver me timbers! Welcome aboard, Jim me hearty! I be happy to chat with ye. What be bringin' ye to these fair digital waters? Got a question or just lookin' to pass the time with a swashbucklin' conversation?


In [18]:
config = {"configurable": {"thread_id": "abc789"}}
query = "Hi I'm Todd, please write an essayi 500 words"
language = "English"

input_messages = [HumanMessage(query)]
for chunk, metadata in app.stream(
    {"messages": input_messages, "language": language},
    config,
    stream_mode="messages",
):
    if isinstance(chunk, AIMessage):  # Filter to just model responses
        print(chunk.content, end="")

Arrrr, shiver me timbers! Ah, Todd me lad, I've got a new assignment for ye! Here be me essay on the impact o' piracy on the development o' the Caribbean region.

Piracy be a scourge that has plagued the Caribbean for centuries, and its impact on the region be far-reaching and profound. From the Golden Age o' Piracy to the present day, piracy has shaped the course o' history in the Caribbean, affecting everything from trade and commerce to politics and culture.

One o' the most significant impacts o' piracy be on the development o' trade and commerce in the Caribbean. Pirates be notorious for their raids on merchant ships, and their attacks be a major cause o' disruption to trade routes and commerce. This be especially true during the Golden Age o' Piracy, when pirates be at the height o' their power and be able to dictate terms to the colonial powers that ruled the region.

But piracy also be a major driver o' economic development in the Caribbean. Pirates be known for their love o' t

In [19]:
import getpass
import os

if not os.environ.get("GROQ_API_KEY"):
  os.environ["GROQ_API_KEY"] = getpass.getpass("Enter API key for Groq: ")

from langchain.chat_models import init_chat_model

llm = init_chat_model("llama3-8b-8192", model_provider="groq")

In [ ]:
from langchain_core.messages import HumanMessage
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import START, MessagesState, StateGraph

# Define a new graph
workflow = StateGraph(state_schema=MessagesState)


# Define the function that calls the model
def call_model(state: MessagesState):
    response = llm.invoke(state["messages"])
    # Update message history with response:
    return {"messages": response}


# Define the (single) node in the graph
workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

# Add memory
memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

In [22]:
config = {"configurable": {"thread_id": "abc123"}}

In [23]:
query = "Hi! I'm Bob."

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()  # output contains all messages in state

================================== Ai Message ==================================

Hi Bob! It's nice to meet you. Is there something I can help you with or would you like to chat?


In [24]:
query = "What's my name?"

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

Your name is Bob!


In [25]:
query = "What's my name?"
config = {"configurable": {"thread_id": "abc234"}}

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

I apologize, but I'm a large language model, I don't have the ability to know your name or any personal information about you. Each time you interact with me, it's a new conversation and I don't retain any information from previous conversations.


In [35]:

load_dotenv()
api_key=os.getenv("GROQ_API_KEY")

model = ChatGroq(model="llama3-8b-8192")

prompt_template = ChatPromptTemplate([ 
    ("system", "You are a friendly and engaging AI assistant that converses naturally with humans. Your goal is to provide helpful, thoughtful, and well-structured responses while maintaining a conversational and personable tone.Answer only to the point"),
    
    MessagesPlaceholder(variable_name="messages"),
])

# Define a new graph
workflow = StateGraph(state_schema=MessagesState)

# Define the function that calls the model
def call_model(state: MessagesState):
    response = model.invoke(state["messages"])
    # prompt = prompt_template.invoke(state)
    # response = model.invoke(prompt)
    return {"messages": response}

workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

config = {"configurable": {"thread_id": "abc123"}}
query="my name is abccd"
input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)

ai_message=output["messages"][-1] # output contains all messages in state
content = ai_message.content
print(content)

Nice to meet you, Abccd! That's a unique name. Is there something I can help you with or would you like to chat?


In [36]:

config = {"configurable": {"thread_id": "abc123"}}
query="what is my name"
input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)

ai_message=output["messages"][-1] # output contains all messages in state
content = ai_message.content
print(content)

Your name is Abccd!


In [ ]:
from langchain.memory import SimpleMemory

# Use SimpleMemory to persist conversation state
memory = SimpleMemory()

# Define a function that appends new responses to the history
def call_model(state: MessagesState):
    prompt = prompt_template.invoke(state)
    response = model.invoke(prompt)

    # Ensure messages history is updated
    state["messages"].append(response)
    
    return {"messages": state["messages"]}

# Modify workflow.compile to include memory persistence
app = workflow.compile( checkpointer=memory)

# Invoke the app multiple times to test persistence
config = {"configurable": {"thread_id": "abc123"}}

query1 = "my name is bob"
input_messages1 = [HumanMessage(query1)]
output1 = app.invoke({"messages": input_messages1}, config)

# query2 = "Can you repeat my name?"
# input_messages2 = [HumanMessage(query2)]
# output2 = app.invoke({"messages": input_messages2}, config)

print(output1["messages"])  # Should retain history from query1


AttributeError: 'SimpleMemory' object has no attribute 'get_next_version'

In [38]:
from transformers import pipeline
import scipy
text="hello i am bob"
synthesiser = pipeline("text-to-speech", "suno/bark")

def textspeech(text):
    speech = synthesiser("Hello, my dog is cooler than you!", forward_params={"do_sample": True})
    scipy.io.wavfile.write("bark_out.wav", rate=speech["sampling_rate"], data=speech["audio"])

textspeech(text)

c:\Users\LENOVO\AppData\Local\Programs\Python\Python310\lib\site-packages\huggingface_hub\file_download.py:140: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\LENOVO\.cache\huggingface\hub\models--suno--bark. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Error while downloading from https://cdn-lfs.hf.co/repos/76/1e/761e5da7cc6a43df30e6091397aa6748ca4c755

KeyboardInterrupt: 